In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
demo = pd.read_csv("mol.csv")

In [7]:
sub=list(demo['subject_id'])

In [2]:
'''demo = np.loadtxt('valid_subjectlist.txt',delimiter='\t',dtype='str')
file = open("valid_subjectlist.txt", "r")
strings = file.readlines()
sub=[]
for i in range(2085):
    sub.append(strings[0].split(',')[i])
'''

In [8]:
L = 383 #timeseries
S = 10   #from which index to start from
data = np.zeros((len(sub),L-S,48));
label = np.zeros((len(sub),));
print(data.shape)

# load all data
idx = 0
err=0
data_all = None

(2085, 373, 48)


In [9]:
for i in range(len(sub)):
    subject_string = 'fmriprep-deri-'+sub[i]
    subject_num = sub[i]

    #harvardoxford (383,48)
    filename_dir = '/home/ubuntu/Stella/MLVU_multimodality/build functional graph/abcd-fmriprep-rs-time/'+subject_string+'/fmriprep/sub-'+subject_num+'/ses-baselineYear1Arm1/func/'
    filename_full = filename_dir+'sub-'+subject_num+'_ses-baselineYear1Arm1_task-rest_run-1_space-MNIPediatricAsym_cohort-4_res-2_desc-preproc_atlas-harvardoxford_timeseries.npy'
    full_sequence = np.load(filename_full).T;  #(48, 383)

    if full_sequence.shape[1] < L:
        continue

    full_sequence = full_sequence[:,S:L];
    z_sequence = stats.zscore(full_sequence, axis=1) # normalization -> z_sequence shape: (48, 383-5)
        
    # adjacency matrix 만드는 코드에 input으로 넣을 data를 만드는 코드
    data[i,:,:] = np.transpose(z_sequence) # subject, timeseries, roi (2085, 378, 48)

In [10]:
data.shape

(2085, 373, 48)

In [11]:
data[0].shape

(373, 48)

In [23]:
abs(np.corrcoef(data[0,:,0], data[0,:,1]))[0][1]

0.6640235365397175

In [12]:
# compute adj matrix
for k in range(data.shape[0]):
    n_regions = 48 # harvardoxford atlas
    A = np.zeros((n_regions, n_regions)) #initialization
    for i in range(n_regions):
        for j in range(i, n_regions):
            if i==j:
                A[i][j] = 1
            else:
                # used abs value as adjacency matrix
                A[i][j] = abs(np.corrcoef(data[k,:,i], data[k,:,j])[0][1]) # get value from corrcoef matrix
                A[j][i] = A[i][j]

    np.save('/home/ubuntu/Stella/MLVU_multimodality/build functional graph/valid_fMRI_con_mat_1210/%s-func_adj_mat.npy' %sub[k], A)

In [25]:
sub[0]

'NDARINVGL08BW0W'

In [28]:
test = np.load('/home/ubuntu/Stella/MLVU_multimodality/build functional graph/valid_fMRI_con_mat_1208/NDARINVGL08BW0W-func_adj_mat.npy')

In [29]:
test.shape

(48, 48)

In [30]:
test

array([[1.        , 0.66402354, 0.93618781, ..., 0.09636118, 0.44413121,
        0.35770869],
       [0.66402354, 1.        , 0.71802415, ..., 0.31808898, 0.04314908,
        0.03498037],
       [0.93618781, 0.71802415, 1.        , ..., 0.10372621, 0.44360081,
        0.27998134],
       ...,
       [0.09636118, 0.31808898, 0.10372621, ..., 1.        , 0.53778901,
        0.53266544],
       [0.44413121, 0.04314908, 0.44360081, ..., 0.53778901, 1.        ,
        0.7219341 ],
       [0.35770869, 0.03498037, 0.27998134, ..., 0.53266544, 0.7219341 ,
        1.        ]])